In [ ]:
from pystac_client import Client
from odc.stac import load, configure_s3_access

In [ ]:
# STAC Catalog URL
catalog = "https://earth-search.aws.element84.com/v1"

# Create a STAC Client
client = Client.open(catalog)

# This line will fail if you don't have credentials configured
_ = configure_s3_access(cloud_defaults=True, requester_pays=True)

In [ ]:
# Find a location you're interested in on Google Maps and copy the coordinates
# by right-clicking on the map and clicking the coordinates

# These coords are in the order Y then X, or Latitude then Longitude
# coords = 20.7748, 106.7785  # Near Haiphong
coords = 12.29308, 109.22544  # Near Phuong Vinh Hoa 
buffer = 0.2
bbox = (coords[1] - buffer, coords[0] - buffer, coords[1] + buffer, coords[0] + buffer)
landsat_stretch = dict(vmin=7500, vmax=18000)

datetime = "2024-06"

In [ ]:
items = client.search(
    collections=["landsat-c2-l2"],
    bbox=bbox,
    datetime=datetime,
    query={"eo:cloud_cover": {"lt": 50}},
).item_collection()

print(f"Found {len(items)} items")

In [ ]:
data = load(
    items,
    bbox=bbox,
    collection="landsat-c2-l2",
    measurements=["red", "green", "blue"],
    groupby="solar_day",
    chunks={"x": 2048, "y": 2048},
)
data

In [ ]:
data.to_array().plot.imshow(col="time", col_wrap=2, size=6, **landsat_stretch)

In [ ]:
best = data.sel(time="2024-06-11").squeeze()
visualisation = best.odc.to_rgba(**landsat_stretch)

visualisation.odc.explore(**landsat_stretch)

In [ ]:
visualisation.odc.write_cog("landsat_example.tif", overwrite=True)